## Scenario 1: Strength and elongation of a dual-phase steel

Lets considder an important class of steels typicall used for automotive applications. They are typically alloyed with C, Mn + other elements to aid hardening. The name "dual" comes from the fact that the microstructure consists of to phases, the soft ferrite phase (F) and the hard martensite phase (M). The balance between these two are controlled first by the annealing temperature where a higher temperature promotes more austenite (A) which is the phase that martensite forms from during cooling. 

- The amount of ferrite during annealing is modelled by the function _ferrite_fraction_during_annealing()_

However martensite is formed at ~400C and austenite may transform back to ferrite during cooling if the cooling rate is to low. In this little example we assume that the cooling rate is always say 100K/s, during such conditions all austenite can transform to martensite _if_ enough hardening elements are in the alloy. 

- How much ferrite that is transformed back to ferrite is modelled by the function _ferrite_formation_during_cooling()_

At that stage we have a dual-phase microstructure at room temperature. The macroscopic properties of this steel grade is mainly governed by the fraction of the two constituents and the alloying element in each phase. The fraction of phases has a dramatic effect of the strength so the ML should ideally be able to capture that. For simplicity lets assume that the elements are evenly distributed between ferrite and martensite _except carbon_. Carbon is a small and mobile element and the solubility is much higher in austenite than in ferrite so at the annealing temperature most carbon will be there. Hence, the smaller the martensite fraction the higher its carbon content. The higher the carbon content the higher its dislocation density and hence its strength. 

- The dislocation density of the martensite is modelled by the function _dd_of_martensite_as_a_fkn_of_C()_
- The ductility measured as the elongation is modelled by the function _longation_as_a_fkn_of_ferrite_fraction()

The steel after annealing is typically referred to as "as-quenched", and we could calculate for example the strength using some variant of rule of mixture. After anneling it is however practise to perform "tempering". This is a treatment at lower tempeature, say 100-500C with the purpose of modifying microstrucure somewhat with the purpose of making the material more ductile but without too much loss in strength. Lets also include that to have more variables for the ML model to explore.

- The function _dislocation_density_tempering()_ is used to model the decrease in dislocation density of martensite during tempering
- The function _elongation_increse_by_tempering()_ is used to model the increase in ductility during tempering. 

The attempt was to find a small sweet-spot where the ductility can be somewhat increased without too much loss in strengh but perhaps one need to modify the parameters a bit. A good exercise is to plot all the functions!

Some parameters used in the model:

Data for steel, taken mainly from Galindo-Nava and Rivera-Diaz-Castillo, Acta Met 2015 [GR2015]. 


sigma_0 = friction stress

rp = mean radius of particles (nm)

fp = volume fraction of particles (-)

G = shear modulus (GPa), referred to as µ i in Galindo-Nava 2015

b = burgers vector (nm). 

HP = Hall-Petch constant (MPa*(µm)^0.5)

d = average grain size (µm)

M = Taylor factor (-)

dd = dislocation density (1/m3)

xYY = mole fraction of element YY (-)

wYY = weight fraction of element YY (-)

betaYY = solid solution strengthening factor for element YY (MPa/atom)

T = temperature (degC)

In [19]:
import numpy as np
from random import random
import pandas as pd

## Strengthening mechanisms

In [20]:
def sigma_p(rp=10, fp=0.01, G=80, b=0.286):
    
    # Orowan bowing particle strengthening
    return 1000*0.26*G*b*np.sqrt(fp)*np.log(rp/b)/rp       # (MPa)

In [21]:
def sigma_hp(HP=300, d=10):
    
    # Hall-Petch grain size hardening
    return HP/np.sqrt(d)                                   # (MPa)

In [22]:
def sigma_dd(alpha=0.25, M=3, G=80, b=0.286, dd=1E15):
    
    # Dislocation-density strengthening accoring to Taylor equation. Alpha taken as 0.
    return 1E-6*alpha*M*G*b*np.sqrt(dd)                     # (MPa)

In [23]:
def sigma_ss_1a(xCr=0.01, xMn=0.02, xMo=0.005, xNi=0.001, xSi=0.01):
    
    betaCr = 434
    betaMn = 213
    betaMo = 2143
    betaNi = 334
    betaSi = 732
    
    # Solid solution strengthening (one of many ways..)
    return np.sqrt(betaCr*betaCr*xCr) + np.sqrt(betaMn*betaMn*xMn) + np.sqrt(betaMo*betaMo*xMo) + np.sqrt(betaNi*betaNi*xNi) + np.sqrt(betaSi*betaSi*xSi)

In [24]:
def sigma_ss(wMn=2E-2, wCr=0.1E-2, wMo=0.1E-2, wNi=0.01E-2, wSi=0.3E-2):
    
    MMn = 54.94; MCr = 52.00; MMo = 95.95; MNi = 55.69; MSi = 28.09 ; MFe = 55.84
    wFe = 1 - wMn - wCr - wMo - wNi - wSi
    
    xMn = (wMn/MMn)/(wMn/MMn + wCr/MCr + wMo/MMo + wNi/MNi + wSi/MSi + wFe/MFe)
    xCr = (wCr/MCr)/(wMn/MMn + wCr/MCr + wMo/MMo + wNi/MNi + wSi/MSi + wFe/MFe)
    xMo = (wMo/MMo)/(wMn/MMn + wCr/MCr + wMo/MMo + wNi/MNi + wSi/MSi + wFe/MFe)
    xNi = (wNi/MNi)/(wMn/MMn + wCr/MCr + wMo/MMo + wNi/MNi + wSi/MSi + wFe/MFe)
    xSi = (wSi/MSi)/(wMn/MMn + wCr/MCr + wMo/MMo + wNi/MNi + wSi/MSi + wFe/MFe)

    betaCr = 434; betaMn = 213; betaMo = 2143; betaNi = 334; betaSi = 732    
    
    # Solid solution strengthening (one of many ways..)
    return np.sqrt(betaCr*betaCr*xCr) + np.sqrt(betaMn*betaMn*xMn) + np.sqrt(betaMo*betaMo*xMo) + np.sqrt(betaNi*betaNi*xNi) + np.sqrt(betaSi*betaSi*xSi)

## Some additional relations to simulate dual phase steels and its elongation


In [25]:
def ferrite_fraction_during_annealing(T=800, wC=0.2E-2, wMn=2E-2, wCr=1E-2, wMo=0.5E-2, wNi=0.1E-2, wSi=1E-2):
    # Variation of the ferrite fraction with annealing temperature
    
    # Trazka 2016, can be done better with Thermo-Calc but not the point here. Somewhat surprising negative Mo for A1
    T_A1 = 742 - 29*wC - 14*wMn + 13*wSi + 16*wCr - 16*wMo
    T_A3 = 925 - 219*np.sqrt(wC*100) - 700*wMn + 3900*wSi - 1600*wNi + 1300*wMo
    
    k = 1/(T_A3 - T_A1)
    m = -T_A1*k
    
    f_aust = T*k + m
    
    if f_aust < 0:
        f_aust = 0
    elif f_aust > 1:
        f_aust = 1

    return 1 - f_aust

In [26]:
def ferrite_formation_during_cooling(wMn=0.02, wCr=0.01, wMo=0.005, wNi=0.001, wSi=0.01):
    # Quick and dirty hardenability model, assuming constant cooling rate say 100K/s it outputs the amount of ferrite
    # formed during cooling with the purpose that that ML model needs to find out that certain level of hardening elements
    # are needed or the ferrite fraction will be too high. 
    
    hardening_factor = 0.75*wCr + 1*wMn + 1*wMo + 0.5*wNi + 0.5*wSi # Resonable values, no reference. 
    # Picking a line that produces 100% ferrite with no hardening and 0% ferrite when hardening factor =>2%
    k = -1/0.02
    m = 1
    
    f_ferrite_during_cooling = k*hardening_factor + m
    if f_ferrite_during_cooling < 0:
        f_ferrite_during_cooling = 0
    
    return f_ferrite_during_cooling

In [27]:
def dd_of_martensite_as_a_fkn_of_C(wC=0.1E-2):
    # Variation of dislocation density of martensite with carbon content
    
    # Model from Morito et al 2003 (rough linear estimate)
    k = (3-0.7E-2)/0.8E-2
    m = 0.7
    
    return (0.7 + k*wC)*1E15

In [28]:
def elongation_as_a_fkn_of_ferrite_fraction(f_ferrite=0.5):
    # Variation of elongation with ferrite fraction for ferrite-martensite dual phase steels
    
    # This one is strictly aftificial i.e. no reference but chosen to be resonable. It results in 2% elongation for 100% martensite / 0% ferrite
    # with rapid increase with ferrite content and saturates at 26%.
    
    return np.sqrt(400*f_ferrite) + 2
    #return 5*np.log(f_ferrite+0.01) - 5*np.log(0.01) + 2    # this one has more rapid increase in elongation for small 

In [29]:
def dislocation_density_tempering(dd=1E15, T=300):
    # Decrese in dislocation density during tempering
    
    # This one is strictly aftificial i.e. no reference but chosen to be resonable. Beyond T_crit there is
    # a quadratic decay
    T_crit = 200
    T_decay = 300
    
    dd_after_tempering = dd - np.heaviside((T-T_crit),1)*dd*((T-T_crit)**2)/(T_decay**2)
    if dd_after_tempering < 0:
        dd_after_tempering = 0
    
    return dd_after_tempering
    

In [30]:
def elongation_increse_by_tempering(f_ferrite=0.5, T=300):
    # Simulate an increase in elongation for near-fully martensitic steels which would motivate tempering 
    
    # This one is strictly aftificial i.e. no reference but chosen to be resonable using a sigmoid time the fraction 
    # of martensite so that the gain in elongation is smaller for low martensite content 
    T_crit = 250
    max_elongation_increse = 5
    rate_of_increase = 0.04
    
    return max_elongation_increse*(1-f_ferrite)**2/(1+np.exp(-rate_of_increase*(T-T_crit)))

## Fetching data 

This snippet uses the function above to calculate mainly strength and elongation of the steel as a function on annealing temperature, tempering temperature and composition

This case considder solid solution hardening (same contribution for each phase). This makes choice of alloying elements a bit more intresting since it also provides hardening, in particular for Mo which has strong solution hardening effect. 

In [31]:
def fetch_data_scenario(x):
    # Numpy array with [Annealing_temperature, Tempering_temperature, wC, wMn, wCr, wMo, wNi, wSi]
    #
    # Annealing temperature in the range 750 - 900C
    # Tempering temperature in the range 100 - 500C
    #
    # w refer to weight fraction of element, for these steels typically 
    # C: 0.0005 - 0.004
    # Mn: 0.005 - 0.03
    # Cr: 0 - 0.01
    # Mo: 0 - 0.01
    # Ni: 0 - 0.01
    # Si: 0 - 0.01
    
    sigma_0_martensite = 400 - 100       #The minus here represent the difference between scenario 1a and 2b
    sigma_0_ferrite = 250 - 100          #The minus here represent the difference between scenario 1a and 2b
    
    
    # Fraction of ferrite and martensite after annealing
    f_ferrite_at_T = ferrite_fraction_during_annealing(T=x[0], wC=x[2], wMn=x[3], wCr=x[4], wMo=x[5], wNi=x[6], wSi=x[7])
    f_ferrite_cooling = ferrite_formation_during_cooling(wMn=x[3], wCr=x[4], wMo=x[5], wNi=x[6], wSi=x[7])
    f_ferrite = f_ferrite_at_T + f_ferrite_cooling
    if f_ferrite > 1:
        f_ferrite = 1
        
    f_martensite = 1 - f_ferrite
    
    # Carbon content in martensite (assume density is the same in ferrite and martensite) and its dislocation density 
    # in as-quenched state
    if f_martensite > 0:
        wC_martensite = x[2]/f_martensite
        dd_martensite_as_q = dd_of_martensite_as_a_fkn_of_C(wC=wC_martensite)
    
    # Tempering effects
    if f_martensite > 0:
        dd_martensite_tempered = dislocation_density_tempering(dd=dd_martensite_as_q, T=x[1])
    
    elongation_increase_tempering = elongation_increse_by_tempering(f_ferrite=f_ferrite, T=x[1])
    
    # Properties - elongation
    elongation = elongation_as_a_fkn_of_ferrite_fraction(f_ferrite) + elongation_increase_tempering
    
    # Phase strength
    # considder only dislocation hardening of martensite for simplicity, other effects baked into sigma_0_martensite
    if f_martensite > 0:
        strength_martensite = sigma_0_martensite + sigma_dd(alpha=0.25, M=3, G=80, b=0.286, dd=dd_martensite_tempered) + sigma_ss(wMn=x[3], wCr=x[4], wMo=x[5], wNi=x[6], wSi=x[7])
    else:
        strength_martensite = 0
    
    strength_ferrite = sigma_0_ferrite + sigma_ss(wMn=x[3], wCr=x[4], wMo=x[5], wNi=x[6], wSi=x[7]) 
    
    # Strength of alloy - note that there are various ways rule of mixture can be applied
    strength_alloy1 = f_ferrite*strength_ferrite + f_martensite*strength_martensite


    ## Here a realistic residual due to experimental error is added

    # Represents a percentage error seldomly surpassing 3 % 
    elongation_error = np.random.standard_normal()/100 + 1
    elongation = elongation*elongation_error

    # Represents an absolute error of seldomly seldomly surpassing 10 MPa 
    strength_alloy1 = strength_alloy1 + np.random.normal(loc=0, scale=3)
    
    return np.array([f_martensite, strength_martensite, strength_alloy1, elongation])

Data generation supplying rest of the functions

In [32]:
def data_generation(n = 1000, seed = 0):
    """
    Generates data.
    ----------
    
    n: integer
        Number of materials to generate.
    seed: integer.
        Sets random seed.
    Returns
    -------
    x_matrix: n x input dim np.array
        Input data.
    y_matrix: n x output dim np.array
        Output data.
    """  
    # Numpy array with [Annealing_temperature, Tempering_temperature, wC, wMn, wCr, wMo, wNi, wSi]
    x_dim = 8
    y_dim = 4
    x_matrix = np.zeros([n,x_dim])
    y_matrix = np.zeros([n,y_dim])

    if seed != 0:
        np.random.seed(seed)

    Annealing_temperature_vec = np.random.randint(765, 845,n)
    Tempering_temperature_vec = np.random.randint(150,300,n)
    wC_vec = np.random.uniform(0.0005, 0.004,n)
    wMn_vec = np.random.uniform(0.005, 0.02,n)
    wCr_vec = np.random.uniform(0, 0.005,n)
    wMo_vec = np.random.uniform(0, 0.005,n)
    wNi_vec = np.random.uniform(0, 0.005,n)
    wSi_vec = np.random.uniform(0, 0.01,n)

    for i in range(n):
        x_matrix[i,:] = np.array([Annealing_temperature_vec[i], Tempering_temperature_vec[i], wC_vec[i], wMn_vec[i], wCr_vec[i], wMo_vec[i], wNi_vec[i], wSi_vec[i]])
        y_matrix[i,:] = fetch_data_scenario(x_matrix[i,:])
    
    return x_matrix, y_matrix

In [33]:
def experiment_data_generation(compositions, n_temp_tests = 2):
    """
    Generates data.
    ----------
    compositions: np.array
        Compositions of training data.
    n_temp_tests: integer.
        number of temperatures to test.
    Returns
    -------
    x_matrix: np.array
        Input data.
    y_matrix: np.array
        Output data.
    """  
    # Numpy array with [Annealing_temperature, Tempering_temperature, wC, wMn, wCr, wMo, wNi, wSi]
    temp_combinations = n_temp_tests**2
    n = compositions.shape[0]*temp_combinations
    
    x_dim = 8
    y_dim = 4
    x_matrix = np.zeros([n,x_dim])
    y_matrix = np.zeros([n,y_dim])

    # Random temperatures
    Annealing_temperature_vec = np.random.randint(765,845,n_temp_tests)
    Tempering_temperature_vec = np.random.randint(150,300,n_temp_tests)

    # Deterministic temperatures
    Annealing_temperature_vec = np.linspace(780, 830, num=n_temp_tests)
    Tempering_temperature_vec = np.linspace(175, 275, num=n_temp_tests)

    Annealing_temperature_vec = np.repeat(Annealing_temperature_vec, n_temp_tests, axis=0)
    Tempering_temperature_vec = np.tile(Tempering_temperature_vec, n_temp_tests).flatten()

    wC_vec = np.repeat(compositions[:,0], temp_combinations, axis=0)
    wMn_vec = np.repeat(compositions[:,1], temp_combinations, axis=0)
    wCr_vec = np.repeat(compositions[:,2], temp_combinations, axis=0)
    wMo_vec = np.repeat(compositions[:,3], temp_combinations, axis=0)
    wNi_vec = np.repeat(compositions[:,4], temp_combinations, axis=0)
    wSi_vec = np.repeat(compositions[:,5], temp_combinations, axis=0)
    
    for i in range(n):
        j = i % temp_combinations
        k = i % temp_combinations
        x_matrix[i,:] = np.array([Annealing_temperature_vec[j], Tempering_temperature_vec[k], wC_vec[i], wMn_vec[i], wCr_vec[i], wMo_vec[i], wNi_vec[i], wSi_vec[i]])
        y_matrix[i,:] = fetch_data_scenario(x_matrix[i,:]) 
    
    return x_matrix, y_matrix

In [34]:
def cost_vec():
    """
    Generates cost vector.
    Returns
    -------
    comp_cost: np.array
        cost vector.
    """ 
    # Cost per kg in USD [C, Mn, Cr, Mo, Ni, Si, Fe]
    comp_cost = np.array([0.17, 0.3, 10.5, 74, 24, 10, 0.15])
    return comp_cost

In [35]:
def fetch_comp_data(filename='realistic_compositions_in_weight_percent.xlsx'):
    """
    Fetch material compositions data.
    ----------
    Fliename: string
        Filename.
    Returns
    -------
    comps: np.array
        Compositions.
    """ 
    comps = pd.read_excel(filename, usecols=list(range(6)))
    comps = np.array(comps)/100
    return comps